In [ ]:
!pip install -qq matplotlib pandas
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
import json
import os

# Intro
We have attached our Jupyter server to three different repositories in our Pachyderm pipline DAG:

- **trips** - This repo is populated with a daily file that records the number of bicycle trips recorded by NYC's citibike bike sharing company on that particular day (data from [here](https://www.citibikenyc.com/system-data)).
- **sales** - This repo includes a single CSV file called `sales.csv`.  `sales.csv` is updated daily by a pipeline that processes each file in `trips` to calculate sales for the day.  Note, here we are using a fictional multiplier, $5/trip, to calculate daily "sales" (i.e., these are not actually the sales figures for citibike).
- **weather** - This repo is populated daily with a JSON file representing the weather forecast for that day from [forecast.io](https://darksky.net/forecast/40.7127,-74.0059/us12/en).

By attaching to these separate points in our DAG (`trips`, `weather`, and `sales`) we can bring our data together at a particular commit (i.e., a particular point in history), without explicitly planning a pipeline stage that takes these repos as input.

# Import sales data from the `sales` repo
Let's first grab our sales data from the `sales` repo and plot data:

In [ ]:
salesDF = pd.read_csv('/pfs/sales/sales.csv', names=['Date', 'Sales'], parse_dates=['Date'], skip_blank_lines=True)
salesDF.dropna(thresh=1, inplace=True)
salesDF.sort_values(by=['Date'], inplace=True)
salesDF = salesDF.reset_index(drop=True)
salesDF.head()

In [ ]:
salesDF.plot(x='Date', y='Sales', figsize=(13, 5))
plt.show()

On 7/30/2016 and 7/31/2016 we can see a sharp drop in sales.

# Import trip data from the `trips` repo, join with the sales data
We can then grab the trip count data from the `trips` repo, put that into a separate dataframe, and merge the two data frames.  This way we can join our data from multiple repos interactively to try and draw some conclusions.

In [ ]:
trip_data = {'Date': [], 'Trips': []}

for fn in os.listdir('/pfs/trips/'):
    if os.path.isfile('/pfs/trips/' + fn):              
        data = pd.read_csv('/pfs/trips/' + fn, parse_dates=['Date'])
        trip_data['Trips'] += data.ix[0]['Trips over the past 24-hours (midnight to 11:59pm)'],
        trip_data['Date'] += data.ix[0]['Date'],

tripsDF = pd.DataFrame(trip_data, columns=['Date', 'Trips'])
tripsDF.dropna(thresh=1, inplace=True)
tripsDF.sort_values(by=['Date'], inplace=True)
tripsDF = tripsDF.reset_index(drop=True)
tripsDF.head()

In [ ]:
dataDF = salesDF.join(tripsDF.set_index('Date'), on='Date')
dataDF.head()

# Import supplemental JSON weather data from the `weather` repo, and join with our other data
Finally, we will pull in the JSON weather data from the `weather` repo and again merge that with the sales and trip data.  We suspect that the weather likely had something to do with the poor sales on the 30th and 31st of July.  In particular we expect that precipitation might have led to the poor sales, so we will extract the daily precipitation probabilities and join that with our previously created dataframe.

In [ ]:
precip_data = {'Date': [], 'Precipitation': []}
for fn in os.listdir('/pfs/weather/'):
    if os.path.isfile('/pfs/weather/' + fn):
        with open('/pfs/weather/' + fn) as data_file:    
            data = json.load(data_file)
        precip_data['Precipitation'] += data['daily']['data'][0]['precipProbability'],
        precip_data['Date'] += pd.to_datetime(fn),
precipDF = pd.DataFrame(precip_data)
precipDF.dropna(thresh=1, inplace=True)
precipDF.sort_values(by=['Date'], inplace=True)
precipDF = precipDF.reset_index(drop=True)
precipDF.head()

In [ ]:
dataDF = dataDF.join(precipDF.set_index('Date'), on='Date')
dataDF.head()

# Visualize the sales in the context of weather
Finally, we confirm our suspicions by visualizing the precipitation probabilities with the sales data:

In [ ]:
%matplotlib notebook
ax = dataDF.plot(x='Date', secondary_y=['Precipitation'], figsize=(13, 8))
ax.set_ylabel('Sales (dollars), # Trips')
ax.right_ax.set_ylabel('Precipitation probability')
ax.right_ax.legend(loc='best')
ax.legend(loc='upper left')
plt.show()

We can see that there was a probability of precipitation in NYC above 70% both of the days in question.  This is likely to be the explanation for the poor sales.  Of course, we can attach our Jupyter notebook to any Pachyderm repos at at commit to explore other unexpected behavior, develop further analyses, etc.